<a href="https://colab.research.google.com/github/engineer-nicolas/cs50sql/blob/master/lecture_3_Writing/lecture_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture 3 - Writing - CS50 SQL harvard

https://cs50.harvard.edu/sql/notes/3/


In this lecture, we’ll explore how to add, update, and delete data in our databases.

The Boston MFA (Museum of Fine Arts) is a century-old museum in Boston. The MFA manages a vast collection of historical and contemporary artifacts and artwork. We will focus now on the creation (or insertion) of data in a Boston MFA database.

Let's create a database called mfa.db.

In [1]:
# Let's import the required libraries
import sqlite3
import pandas as pd

# Connect to the database file
# it will create the file if it doesn't exist
conn = sqlite3.connect('mfa.db')
# MFA stands for the Boston Museum of Fine Arts (MFA)


In [2]:
conn.execute("""
CREATE TABLE "collections" (
    "id" INTEGER,
    "title" TEXT NOT NULL,
    "accession_number" TEXT NOT NULL UNIQUE,
    "acquired" NUMERIC,
    PRIMARY KEY("id")
);
""")

# Commit changes to save the table in the database
conn.commit()

# Let's inspect the schema of the collections table
pd.read_sql_query(
    "PRAGMA table_info(collections);",
    conn
)

,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,1
1,1,title,TEXT,1,None,0
2,2,accession_number,TEXT,1,None,0
3,3,acquired,NUMERIC,0,None,0


## INSERT INTO


This SQL statement is used to insert a row of data into a given table.



In [3]:
# Execute the INSERT statement
conn.execute('''
INSERT INTO "collections" ("id", "title", "accession_number", "acquired")
VALUES (1, 'Profusion of flowers', '56.257', '1956-04-12');
''')

# Save changes to the database
conn.commit()

pd.read_sql_query('SELECT * FROM collections;', conn)

,id,title,accession_number,acquired
0,1,Profusion of flowers,56.257,1956-04-12


This command requires the list of columns in the table that will receive new data and the values to be added to each column, in the *same order*.

IMPORTANT: SQLite can fill out the primary key values automatically. To make use of this functionality, we omit the ID column altogether while inserting a row.

In [4]:
# Execute the INSERT statement
conn.execute('''
INSERT INTO "collections" ("title", "accession_number", "acquired")
VALUES ('Farmers working at dawn', '11.6152', '1911-08-03');
''')

# Save changes to the database
conn.commit()

pd.read_sql_query('SELECT * FROM collections;', conn)

,id,title,accession_number,acquired
0,1,Profusion of flowers,56.257,1956-04-12
1,2,Farmers working at dawn,11.6152,1911-08-03


SQLite selects the highest primary key value in the table and increments it to generate the next primary key value — in this case, 1.

### Inserting Multiple Rows

Let us now insert two new paintings into the collections table.

In [5]:
# Execute the INSERT statement
conn.execute('''
INSERT INTO "collections" ("title", "accession_number", "acquired")
VALUES
('Imaginative landscape', '56.496', NULL),
('Peonies and butterfly', '06.1899', '1906-01-01');
''')

# Save changes to the database
conn.commit()

pd.read_sql_query('SELECT * FROM collections;', conn)

,id,title,accession_number,acquired
0,1,Profusion of flowers,56.257,1956-04-12
1,2,Farmers working at dawn,11.6152,1911-08-03
2,3,Imaginative landscape,56.496,None
3,4,Peonies and butterfly,06.1899,1906-01-01


## IMPORT CSV FILE

SQLite makes it possible to import a CSV file directly into our database. We can import the CSV by running a SQLite command:

`.import --csv --skip 1 mfa.csv collections`

The first argument, --csv indicates to SQLite that we are importing a CSV file.

The second argument indicates that the first row of the CSV file (the header row) needs to be skipped, or not inserted into the table.




## INSERT INTO ___ SELECT ___ FROM - Inserting from another table

In [10]:
csv_file = """title,accession_number,acquired
Profusion of flowers,56.257,1956-04-12
Farmers working at dawn,11.6152,1911-08-03
Spring ,14.769,1914-01-08
Red landscape,56.476,
Peonies and butterfly,06.1899,1906-01-01
"""

with open("temp.csv", "w", encoding="utf-8") as f:
    f.write(csv_file)

# Read the CSV file into a DataFrame
df = pd.read_csv("temp.csv")
print(df)

                     title  accession_number    acquired
0     Profusion of flowers           56.2570  1956-04-12
1  Farmers working at dawn           11.6152  1911-08-03
2            Spring outing           14.7600  1914-01-08
3    Imaginative landscape           56.4960            
4    Peonies and butterfly            6.1899  1906-01-01


In [14]:
conn.execute('''
CREATE TABLE IF NOT EXISTS temp (
    title TEXT,
    accession_number TEXT,
    acquired NUMERIC
);
''')
conn.commit()

df.to_sql(
    'temp',
    conn,
    if_exists='append',
    index=False
)

5

In [15]:
pd.read_sql_query('SELECT * FROM temp;', conn)


,title,accession_number,acquired
0,Profusion of flowers,56.257,1956-04-12
1,Farmers working at dawn,11.6152,1911-08-03
2,Spring outing,14.76,1914-01-08
3,Imaginative landscape,56.496,
4,Peonies and butterfly,6.1899,1906-01-01
5,Profusion of flowers,56.257,1956-04-12
6,Farmers working at dawn,11.6152,1911-08-03
7,Spring outing,14.76,1914-01-08
8,Imaginative landscape,56.496,
9,Peonies and butterfly,6.1899,1906-01-01


In [16]:
conn.execute("""
INSERT OR IGNORE INTO "collections" ("title", "accession_number", "acquired")
SELECT "title", "accession_number", "acquired"
FROM "temp";
""")

conn.commit()

pd.read_sql_query(
    'SELECT * FROM collections;',
    conn
)


,id,title,accession_number,acquired
0,1,Profusion of flowers,56.257,1956-04-12
1,2,Farmers working at dawn,11.6152,1911-08-03
2,3,Imaginative landscape,56.496,None
3,4,Peonies and butterfly,06.1899,1906-01-01
4,5,Spring outing,14.76,1914-01-08
5,6,Peonies and butterfly,6.1899,1906-01-01


## DELETE

Running the following command would delete all rows from the table `collections`

In [ ]:
conn.execute('''
DELETE FROM "collections";
''')
conn.commit()
pd.read_sql_query('SELECT * FROM collections;', conn)

,id,title,accession_number,acquired


In [ ]:
conn.execute('''
DELETE FROM "collections"
WHERE "title" = 'Spring outing';
''')
conn.commit()
pd.read_sql_query('SELECT * FROM collections;', conn)

,id,title,accession_number,acquired


In [ ]:
conn.execute('''
DELETE FROM "collections"
WHERE "acquired" IS NULL;
''')
conn.commit()
pd.read_sql_query('SELECT * FROM collections;', conn)

,id,title,accession_number,acquired


In [ ]:
conn.execute('''
DELETE FROM "collections"
WHERE "acquired" < '1909-01-01';
''')
conn.commit()
pd.read_sql_query('SELECT * FROM collections;', conn)

,id,title,accession_number,acquired


## UPDATE

Here is the syntax of the update command:

UPDATE table
SET column0 = value0, ...
WHERE condition;

## Triggers

A trigger is a SQL statement that runs automatically in response to another SQL statement, such as an INSERT, UPDATE, or DELETE.

Triggers are useful for maintaining data consistency and automating tasks across related tables.

Let's create a new `transactions` table.

In [ ]:
conn.execute("""
CREATE TABLE "transactions" (
    "id" INTEGER,
    "title" TEXT,
    "action" TEXT,
    PRIMARY KEY("id")
);
""")

# Commit changes to save the table in the database
conn.commit()
# Let's inspect the schema of the transactions table
pd.read_sql_query(
    "PRAGMA table_info(transactions);",
    conn
)

,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,1
1,1,title,TEXT,0,None,0
2,2,action,TEXT,0,None,0


## CREATE TRIGGER ___ BEFORE ___ ON___ BEGIN ___ END; + DELETE/UPDATE + OLD

IMPORTANT: `OLD` is a special keyword that refers to the row being deleted or updated. If this row had a title column, `OLD."title"` would access that column’s value.

When artwork is sold and deleted from `collections`, we want it automatically logged in `transactions` with an action of “sold”.

In [ ]:
# Create the trigger
conn.execute('''
CREATE TRIGGER "sell"
BEFORE DELETE ON "collections"
BEGIN
    INSERT INTO "transactions" ("title", "action")
    VALUES (OLD."title", 'sold');
END;
''')

# Save changes
conn.commit()
pd.read_sql_query("""
SELECT *
FROM sqlite_master
WHERE type = 'trigger';
""", conn)

,type,name,tbl_name,rootpage,sql
0,trigger,sell,collections,0,"CREATE TRIGGER ""sell""\nBEFORE DELETE ON ""colle..."


Let’s test the trigger:

In [ ]:
# Execute the INSERT statement
conn.execute("""
INSERT INTO "collections" ("title", "accession_number", "acquired")
VALUES
('Flowers', '56.493', NULL);
""")

# Execute the DELETE statement
conn.execute("""
DELETE FROM "collections"
WHERE "title" = 'Flowers';
""")
# Save changes to the database
conn.commit()

In [ ]:
pd.read_sql_query("SELECT * FROM transactions;", conn)

,id,title,action
0,1,Flowers,sold
1,2,Flowers,sold


## CREATE TRIGGER ___ AFTER ___ ON ___ + INSERT/UPDATE + NEW

IMPORTANT: `NEW` is a special keyword that refers to the row being inserted or updated. If this row has a title column, `NEW."title"` would accesses that column’s value.


When artwork is bought (inserted into collections), we want it logged in transactions with an action of “bought”.

In [ ]:
# Create the trigger
conn.execute("""
CREATE TRIGGER "buy"
AFTER INSERT ON "collections"
BEGIN
    INSERT INTO "transactions" ("title", "action")
    VALUES (NEW."title", 'bought');
END;
""")

# Save changes
conn.commit()
pd.read_sql_query("""
SELECT *
FROM sqlite_master
WHERE type = 'trigger';
""", conn)

,type,name,tbl_name,rootpage,sql
0,trigger,sell,collections,0,"CREATE TRIGGER ""sell""\nBEFORE DELETE ON ""colle..."
1,trigger,buy,collections,0,"CREATE TRIGGER ""buy"" \nAFTER INSERT ON ""collec..."


Let’s test the trigger:

In [ ]:
# Execute the INSERT statement
conn.execute("""
INSERT INTO "collections" ("title", "accession_number", "acquired")
VALUES
('Flowers', '56.493', NULL);
""")

# Execute the DELETE statement
conn.execute("""
DELETE FROM "collections"
WHERE "title" = 'Flowers';
""")
# Save changes to the database
conn.commit()

In [ ]:
pd.read_sql_query("SELECT * FROM transactions;", conn)

,id,title,action
0,1,Flowers,sold
1,2,Flowers,sold
2,3,Flowers,bought
3,4,Flowers,sold
4,5,Flowers,bought
5,6,Flowers,sold


## Soft Deletions

Soft deletion (or a soft delete) means marking data as deleted rather than actually removing it from the database.


For example, we could add a `deleted` column to the `collections` table with a default value of 0. To “delete” a row, we would update the deleted column to 1. This way, data can be recovered if needed and maintains a complete historical record.

In [ ]:
# Add "deleted" column
conn.execute('''
ALTER TABLE "collections"
ADD COLUMN "deleted" INTEGER DEFAULT 0;
''')

# Save changes to the database
conn.commit()
pd.read_sql_query('SELECT * FROM collections;', conn)

,id,title,accession_number,acquired,deleted
0,1,Profusion of flowers,56.257,1956-04-12,0
1,2,Farmers working at dawn,11.6152,1911-08-03,0
2,3,Imaginative landscape,56.496,None,0
3,4,Peonies and butterfly,06.1899,1906-01-01,0


To “delete” a row, we would update the deleted column to 1:

In [ ]:
# Update "deleted" column without deleting row
conn.execute('''
UPDATE "collections"
SET "deleted"=1
WHERE "title"='Farmers working at dawn';
''')

# Save changes to the database
conn.commit()
pd.read_sql_query('SELECT * FROM collections;', conn)

,id,title,accession_number,acquired,deleted
0,1,Profusion of flowers,56.257,1956-04-12,0
1,2,Farmers working at dawn,11.6152,1911-08-03,1
2,3,Imaginative landscape,56.496,None,0
3,4,Peonies and butterfly,06.1899,1906-01-01,0


Then, to query only non-deleted rows:

In [ ]:
pd.read_sql_query("""
SELECT * FROM "collections"
WHERE "deleted" != 1;
""", conn)

,id,title,accession_number,acquired,deleted
0,1,Profusion of flowers,56.257,1956-04-12,0
1,3,Imaginative landscape,56.496,None,0
2,4,Peonies and butterfly,06.1899,1906-01-01,0


This way, data can be recovered if needed and maintains a complete historical record.